In [101]:
#import findspark, pyspark
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import *
from datetime import datetime
from functools import reduce
from pyspark.sql.functions import col

#sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

file_attack = 'gtd_14to17_0718dist'
file_airport = 'airports-extended'
file_largest_airport = 'largest-global-airports-passenger-traffic'
file_passengers = 'avia_par_be'

# Fetch Data

In [102]:
#data_path = 'D:/DataMining/taba/'
data_path = '../data/Datasets/'

attack_data = sc.textFile(data_path + 'gtd-data/' + file_attack + '.csv')
airport_data = sc.textFile(data_path + file_airport + '.csv')
largest_airport_data = sc.textFile(data_path + file_largest_airport + '.xlsx')
passenger_data = sc.textFile(data_path + file_passengers + '.xls')

In [103]:
# ALT WAY DOES NOT WORK YET QUESTION
#df2 = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load(data_path + 'gtd-data/' + file_attack + '.csv')
#df2.printSchema()
#df2.select(df2.country_txt).show(1)
#df2.select(col('country_txt')).show(1)
# https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
# => fill null values in dataframe -> fillna()

# Prepare Attack Data

In [104]:
# Define columns for dataframe
new_columns = attack_data.first()
new_columns = new_columns.split(";")
attack_data = attack_data.filter(lambda l: l != new_columns)
df_terror_data = attack_data.map(lambda x: x.split(';')).toDF()
old_columns = df_terror_data.schema.names

In [105]:
# apply new column names
df_terror_data = reduce(lambda data, idx: data.withColumnRenamed(old_columns[idx], new_columns[idx]), range(len(old_columns)), df_terror_data)

# Query Attack Data

In [106]:
#query_result = df_terror_data.filter(col("country_txt") == "Belgium").select('iyear', 'imonth', 'iday','country_txt', 'summary')
query_result = df_terror_data.filter(df_terror_data.country_txt == "Belgium").select('iyear', 'imonth', 'iday','country_txt', 'summary')

In [107]:
query_result.show(10)

+-----+------+----+-----------+--------------------+
|iyear|imonth|iday|country_txt|             summary|
+-----+------+----+-----------+--------------------+
| 2014|     5|  24|    Belgium|05/24/2014: Assai...|
| 2014|     9|  16|    Belgium|09/16/2014: Assai...|
| 2016|     3|  22|    Belgium|03/22/2016: Two s...|
| 2016|     3|  22|    Belgium|03/22/2016: A sui...|
| 2016|     8|   6|    Belgium|08/06/2016: An as...|
| 2016|     8|  29|    Belgium|08/29/2016: Assai...|
| 2016|    10|   5|    Belgium|10/05/2016: An as...|
| 2016|    12|  23|    Belgium|12/23/2016: Secur...|
| 2017|     5|   1|    Belgium|"05/01/2017: An a...|
| 2017|     6|  20|    Belgium|"06/20/2017: A su...|
+-----+------+----+-----------+--------------------+
only showing top 10 rows



In [111]:
query_result.select('summary').show(10, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|summary                                                                                                                                                                                                                                                                      

# Prepare airport_data

In [109]:
fields = []
fields.append(StructField('airport_id', StringType(), True)) # QUESTION IntegerType() does not work even when parsed
fields.append(StructField('airport_name', StringType(), True))
fields.append(StructField('city', StringType(), True))
fields.append(StructField('country', StringType(), True))
fields.append(StructField('IATA', StringType(), True)) # kind of airport identifier
fields.append(StructField('ICAO', StringType(), True)) # different kind of airport identifier
fields.append(StructField('latitude', StringType(), True))
fields.append(StructField('longitude', StringType(), True))
fields.append(StructField('unknown1', StringType(), True))
fields.append(StructField('unknown2', StringType(), True))
fields.append(StructField('unknown3', StringType(), True))
fields.append(StructField('continent_city', StringType(), True))
fields.append(StructField('idk_column', StringType(), True))
fields.append(StructField('source', StringType(), True))
schema = StructType(fields)

#clean_data = (airport_data
#              .map(lambda line: line.split(';'))
#              .map(lambda line: int(line[0]))
#             )
df_airport_data = sqlContext.createDataFrame(airport_data.map(lambda line: line.split(';')), schema)

# Query airport_data

In [110]:
(df_airport_data
 .select(df_airport_data.airport_name, df_airport_data.city,df_airport_data.continent_city)
 .filter(df_airport_data.continent_city == 'Europe/Brussels')
 .show(df_airport_data.count(),False))

+--------------------------------------+-----------------+---------------+
|airport_name                          |city             |continent_city |
+--------------------------------------+-----------------+---------------+
|Antwerp International Airport (Deurne)|Antwerp          |Europe/Brussels|
|Beauvechain Air Base                  |Beauvechain      |Europe/Brussels|
|Kleine Brogel Air Base                |Kleine Brogel    |Europe/Brussels|
|Brussels Airport                      |Brussels         |Europe/Brussels|
|Jehonville Air Base                   |Bertrix          |Europe/Brussels|
|Brussels South Charleroi Airport      |Charleroi        |Europe/Brussels|
|Chièvres Air Base                     |Chievres         |Europe/Brussels|
|Koksijde Air Base                     |Koksijde         |Europe/Brussels|
|Florennes Air Base                    |Florennes        |Europe/Brussels|
|Wevelgem Airport                      |Kortrijk-vevelgem|Europe/Brussels|
|Liège Airport           

# Prepare largest_airport_data

# Prepare passenger_data